In [1]:
import pandas as pd
import numpy as np
import os
import time
from IPython.display import clear_output

indivs_columns = ["CycleIndivs", "FECTransID", "ContribID", "Contrib", "RecipID", "Orgname", "UltOrg", "RealCode", "Date",
             "Amount", "Street", "City", "State", "Zip", "RecipCodeIndivs", "Type", "CmteID", "OtherID", "Gender", "Microfilm",
             "Occupation", "Employer", "Source"]

cands_columns = ['CycleCands','FECCandID','CID','FirstLastP','Party','DistIDRunFor','DistIDCurr','CurrCand','CycleCand','CRPICO','RecipCodeCands','NoPacs']
cmte_columns = ['CycleCmte','CmteID','PACShort','Affiliate','Ultorg','RecipID','RecipCode','FECCandID','Party','PrimCode','Source','Sensitive','Foreign','Active']

### Set up files and directories

In [2]:
#######################################
#
# Change me
# Enter path to opensecrets root dir here 
#
#######################################

path = os.curdir

In [3]:
def create_path_links(rootname = None, rootpath = None):  
  lst = []
  for root, dirs, files in os.walk(rootpath):
    for file in files:
            if file.startswith(rootname):
              lst.append(os.path.join(root,file))
  return lst

def pre_process_chunk(chunk, filename):
    column_dict = {'indivs': indivs_columns, 'cands': cands_columns} 
    
    #this depends on file path format, maybe theres more effecient way to do this
    filename = filename.split('\\')[-1]
    filename = filename.split('/')[-1]
    filename = filename.split('.')[0]

    chunk.columns = column_dict[filename[:-2]]

    #convert contribID and Recipid to object type
    chunk['ContribID'] = chunk['ContribID'].astype('object')
    chunk['RecipID'] = chunk['RecipID'].astype('object')
    
    #clean individual ids and recip ids so we can remove blank ones
    chunk['ContribID'] = chunk['ContribID'].str.strip()
    chunk['RecipID'] = chunk['RecipID'].str.replace(' ', '')
    chunk['ContribID'] = chunk['ContribID'].str.replace(' ', '')

    #remove rows with blank ids by changing it to nans first
    chunk = chunk.replace(r'^\s*$', np.nan, regex=True)
    chunk = chunk.dropna(subset=['RecipID', 'ContribID'])

    chunk['Date'] = pd.to_datetime(chunk['Date'], format='%m/%d/%Y')
    
    return chunk

def process_summary(chunk):

    #replace nans with 'U' in column 'Gender'
    chunk['Gender'] = chunk['Gender'].fillna('U')
    #1 for male donors and 0 for others
    chunk['Gender'] = chunk['Gender'].str.lower()   
    chunk['MaleDonor'] = chunk['Gender'].str.contains('m').astype(int)
    chunk['FemaleDonor'] = chunk['Gender'].str.contains('f').astype(int)

    #summarize by year, contribID and RecipID return min max dates, total amount and total gift count
    agg = {
        'Date': ['min', 'max'],
        'Amount': ['sum'],
        'FECTransID': ['nunique'],
        'MaleDonor': ['max'],
        'FemaleDonor': ['max']
    }
    df = chunk.groupby(['CycleIndivs','ContribID','RecipID']).agg(agg).reset_index()

    return df

def post_process(chunk):
  #reduce and collect chunks into one final groupby once all files have been processed
  chunk.columns = ['CycleIndivs', 'ContribID', 'RecipID', 'DateMin', 'DateMax', 'Amount', 'GiftCount','MaleDonor','FemaleDonor']
  chunk= chunk.reset_index(drop=True)  

  agg = {'DateMin': 'min',
  'DateMax': 'max',
  'Amount': 'sum',
  'GiftCount': 'sum',
  'MaleDonor': 'max',
  'FemaleDonor': 'max'
    }
 
  chunk = chunk.groupby(['CycleIndivs','ContribID','RecipID']).agg(agg).reset_index()

  return chunk  

def combine_individual_contributions(df):
   
  #count multicycle donors
  df['MultiCycleDonor'] = df.groupby(['ContribID','RecipID'])['CycleIndivs'].transform('nunique') > 1  
  df['MultiCycleDonor'] = df['MultiCycleDonor'].astype(int)

  #count repeat donors
  df['RepeatDonor'] = df.groupby(['ContribID','RecipID'])['GiftCount'].transform('sum') > 1
  df['RepeatDonor'] = df['RepeatDonor'].astype(int)

  agg = {'DateMin': 'min',
  'DateMax': 'max',
  'Amount': 'sum',
  'GiftCount': 'sum',
  'MultiCycleDonor': 'max',
  'RepeatDonor': 'max',
  'MaleDonor': 'max',
  'FemaleDonor': 'max'
    }
    
  df = df.groupby(['CycleIndivs','ContribID','RecipID']).agg(agg).reset_index()

 

  return df


In [4]:
def process_file(filename, chunksize = 5000):  
    df = pd.DataFrame() 
    begin = time.time()
    performance = {}
    for i,chunk in enumerate(pd.read_csv(filename, sep=',', quotechar='|', header=None, encoding='ISO-8859-1', chunksize=chunksize)):

        chunk = pre_process_chunk(chunk, filename)
        chunk = process_summary(chunk)
        
        df = df.append(chunk)       

    
        #performance tracking
        if i % 10 == 0:
            #print records processed
            print('chunk ' + str(i) + ' processed' + ' of file ' + filename.split("\\")[-1])            
            print('{:,}'.format(len(df)) + ' records processed')            
            print('running time: ' + str(round((time.time() - begin)/60, 2)) + ' minutes')           
        clear_output(wait=True)
        
    performance['file'] = filename    
    performance['begin_time'] = begin
    performance['end_time'] = time.time()
    performance['total_time_seconds'] = performance['end_time'] - performance['begin_time']    
    performance['total_time_minutes'] = performance['total_time_seconds']/60
    performance['records_processed'] = len(df)    
    performance['memory_usage_mb'] = df.memory_usage(deep=True).sum()/(1024*1024)   
    performance['memory_usage_gb'] = performance['memory_usage_mb']/1024
    
    
    return df, performance

### Run individual contribution files

In [5]:
indiv_files = create_path_links('indivs', path)
indiv_files

['.\\indivs10.txt',
 '.\\indivs12.txt',
 '.\\indivs14.txt',
 '.\\indivs16.txt',
 '.\\indivs18.txt']

In [6]:
df_indivs = pd.DataFrame()
performance = []
for file in indiv_files:
    print(file)
    #append to df_indivs
    df_tmp, perf = process_file(file)
    df_tmp = post_process(df_tmp)
    performance.append(perf)
    #print performance
    df_indivs = df_indivs.append(df_tmp)   
    del df_tmp

chunk 3300 processed of file indivs18.txt
14,192,719 records processed
running time: 26.39 minutes


In [7]:
df_performance = pd.DataFrame(performance)
df_performance

,file,begin_time,end_time,total_time_seconds,total_time_minutes,records_processed,memory_usage_mb,memory_usage_gb
0,.\indivs10.txt,1.668916e+09,1.668916e+09,72.530879,1.208848,1998598,362.704826,0.354204
1,.\indivs12.txt,1.668916e+09,1.668916e+09,157.800727,2.630012,3565887,647.025311,0.631861
2,.\indivs14.txt,1.668916e+09,1.668916e+09,79.543001,1.325717,2034942,369.267916,0.360613
3,.\indivs16.txt,1.668916e+09,1.668918e+09,1519.262695,25.321045,13116430,2378.399240,2.322656
4,.\indivs18.txt,1.668918e+09,1.668920e+09,1585.038023,26.417300,14198195,2574.836625,2.514489


In [8]:
#collapse all chunks into one final groupby
df_indivs = combine_individual_contributions(df_indivs)
df_indivs.head()


,CycleIndivs,ContribID,RecipID,DateMin,DateMax,Amount,GiftCount,MultiCycleDonor,RepeatDonor,MaleDonor,FemaleDonor
0,2010,L,C00141812,2010-03-24,2010-03-24,2000,1,0,0,0,0
1,2010,U00000000011,C00010603,2010-10-22,2010-10-22,24900,1,1,1,1,0
2,2010,U00000000011,C00042366,2010-06-23,2010-06-23,25000,1,0,0,1,0
3,2010,U00000000011,C00104471,2009-03-31,2009-03-31,10000,1,1,1,1,0
4,2010,U00000000011,C00363994,2009-01-14,2009-01-14,5000,1,0,0,1,0


### Group by Cycle and CandidateID

In [9]:
#aggregate all columns except for the date columns
agg = {'Amount': ['sum', 'mean'],
    'GiftCount': ['sum', 'mean'],
    'ContribID': ['nunique'],
    'MultiCycleDonor': 'sum',
    'RepeatDonor': 'sum',
    'MaleDonor': 'sum',
    'FemaleDonor': 'sum'}

df_indivs = df_indivs.groupby(['CycleIndivs','RecipID']).agg(agg).reset_index()

In [10]:
df_indivs.head()

CycleIndivs    RecipID  Amount              GiftCount           ContribID  \
                             sum         mean       sum      mean   nunique   
0        2010  C00000059    1200  1200.000000         1  1.000000         1   
1        2010  C00000422  582961   725.978829      1046  1.302615       803   
2        2010  C00000638    5025   418.750000        14  1.166667        12   
3        2010  C00000729  473610   668.940678      1074  1.516949       708   
4        2010  C00000885  327532   454.274619      1189  1.649098       721   

  MultiCycleDonor RepeatDonor MaleDonor FemaleDonor  
              sum         sum       sum         sum  
0               0           0         1           0  
1             371         414       572         154  
2               7           7        11           0  
3             509         551       501         169  
4             521         588       626          37

In [11]:
#rename columns
df_indivs.columns = ['Cycle', 'RecipID', 'TotalGiftAmt', 
'TotalGiftAvg', 'TotalGiftCount', 'TotalGiftAvg', 'DonorCount', 
'MultiCycleDonorCount', 'RepeatDonorCount','MaleDonorCount','FemaleDonorCount']

df_indivs.head()

,Cycle,RecipID,TotalGiftAmt,TotalGiftAvg,TotalGiftCount,TotalGiftAvg,DonorCount,MultiCycleDonorCount,RepeatDonorCount,MaleDonorCount,FemaleDonorCount
0,2010,C00000059,1200,1200.000000,1,1.000000,1,0,0,1,0
1,2010,C00000422,582961,725.978829,1046,1.302615,803,371,414,572,154
2,2010,C00000638,5025,418.750000,14,1.166667,12,7,7,11,0
3,2010,C00000729,473610,668.940678,1074,1.516949,708,509,551,501,169
4,2010,C00000885,327532,454.274619,1189,1.649098,721,521,588,626,37


### Process candidate opensecret files

---

In [64]:
candidate_files = ['cands10.txt', 'cands12.txt', 'cands14.txt', 'cands16.txt', 'cands18.txt']

cands = pd.DataFrame()
#read and concat all candidate files into one dataframe
for file in candidate_files:
    #append to cands dataframe
    
    cand = pd.read_csv(file, sep=',', quotechar='|', names=cands_columns, encoding='ISO-8859-1')
    #append to cands
    cands = cands.append(cand)

#only keep candidate where it is running in the current cycle. This will eliminate some duplicate rows
cands = cands[cands['CycleCand'] == 'Y']
cands.head()


,CycleCands,FECCandID,CID,FirstLastP,Party,DistIDRunFor,DistIDCurr,CurrCand,CycleCand,CRPICO,RecipCodeCands,NoPacs
0,2010,H0AK00089,N00031081,Harry Crawford (D),D,AK01,,Y,Y,C,DL,
1,2010,H0AK00097,N00032846,John R Cox (R),R,AK01,,,Y,C,RL,
2,2010,H0AK01038,N00031713,Sheldon Fisher (R),R,AK01,,,Y,C,RL,
3,2010,H0AL00016,N00030909,Martha Bozeman (D),D,AL07,,,Y,O,DL,
4,2010,H0AL01030,N00030621,Peter Gounares (R),R,AL01,,,Y,C,RL,


In [65]:
#if currcand = 'Y' then 1 else 0
cands['curr_cand'] = np.where(cands['CurrCand'] == 'Y', 1, 0)
cands['incumbent'] = np.where(cands['CRPICO'] == 'I', 1, 0)
cands['challenger'] = np.where(cands['CRPICO'] == 'C',1,0)
cands['open_office'] = np.where(cands['CRPICO'] == 'O', 1, 0)
cands['nopacs'] = np.where(cands['NoPacs'] == 'Y', 1, 0)
cands['republican'] = np.where(cands['Party'] == 'R', 1, 0)
cands['democrat'] = np.where(cands['Party'] == 'D', 1, 0)
cands['other_party'] = np.where(cands['Party'].isin(['R','D']), 0, 1)

cands.head()

,CycleCands,FECCandID,CID,FirstLastP,Party,DistIDRunFor,DistIDCurr,CurrCand,CycleCand,CRPICO,RecipCodeCands,NoPacs,curr_cand,incumbent,challenger,open_office,nopacs,republican,democrat,other_party
0,2010,H0AK00089,N00031081,Harry Crawford (D),D,AK01,,Y,Y,C,DL,,1,0,1,0,0,0,1,0
1,2010,H0AK00097,N00032846,John R Cox (R),R,AK01,,,Y,C,RL,,0,0,1,0,0,1,0,0
2,2010,H0AK01038,N00031713,Sheldon Fisher (R),R,AK01,,,Y,C,RL,,0,0,1,0,0,1,0,0
3,2010,H0AL00016,N00030909,Martha Bozeman (D),D,AL07,,,Y,O,DL,,0,0,0,1,0,0,1,0
4,2010,H0AL01030,N00030621,Peter Gounares (R),R,AL01,,,Y,C,RL,,0,0,1,0,0,1,0,0


In [66]:
RecipCode = {'R': 'Republican',
'D': 'Democrat',
'3': 'Other', 
'W': 'Win', 
'L': 'Lose', 
'I': 'Incumbent', 
'C': 'Challenger', 
'O': 'Open Seat', 
'N': 'NonIncumbent'}

In [67]:
#candidate party based on recipCode
cands['cand_party'] = cands['RecipCodeCands'].str[0]
cands['cand_party'] = cands['cand_party'].map(RecipCode)

#candidate status based on recipCode
cands['cand_status'] = cands['RecipCodeCands'].str[1]
cands['cand_status'] = cands['cand_status'].map(RecipCode)

#break out DistIDRunFor into district and state
cands['run_for_state'] = cands['DistIDRunFor'].str[0:2]
cands['run_for_district'] = cands['DistIDRunFor'].str[2:4]

#presidential year if in 2012 or 2016
cands['presidential_year'] = np.where(cands['CycleCands'].isin([2012,2016]), 1, 0)

#if cand_status = win then 1 else 0
cands['label'] = np.where(cands['cand_status'] == 'Win', 1, 0)

In [72]:
cands_out = pd.merge(df_indivs,cands, left_on=['RecipID','Cycle'], right_on=['CID','CycleCands'], how='inner')
cands_out.head()

,Cycle,RecipID,TotalGiftAmt,TotalGiftAvg,TotalGiftCount,TotalGiftAvg,DonorCount,MultiCycleDonorCount,RepeatDonorCount,MaleDonorCount,...,nopacs,republican,democrat,other_party,cand_party,cand_status,run_for_state,run_for_district,presidential_year,label
0,2010,N00000010,509944,1265.369727,565,1.401985,403,57,126,264,...,0,1,0,0,Republican,Win,IN,05,0,1
1,2010,N00000036,5570090,788.517837,12854,1.819649,7064,2205,3656,3960,...,0,0,1,0,Democrat,Lose,WI,S2,0,0
2,2010,N00000078,2058820,1371.632245,2215,1.475683,1501,434,661,886,...,0,0,1,0,Democrat,Win,NY,14,0,1
3,2010,N00000133,91954,947.979381,142,1.463918,97,37,46,62,...,0,0,1,0,Democrat,Win,VI,00,0,1
4,2010,N00000143,383949,876.595890,674,1.538813,438,154,250,296,...,0,0,1,0,Democrat,Win,MA,01,0,1


In [73]:
cands_out.to_csv('candidate_summary_with_labels.csv', index=False)